In [1]:
import numpy as np
import pandas as pd

from os.path import join, exists
from os import makedirs
from glob import glob
import re

from tqdm import tqdm

import scipy.io

import matplotlib.pyplot as plt
import seaborn as sns

from matplotlib import patches
from matplotlib import animation, rc

In [5]:
dir_root = '/mnt/ext5/GA'
dir_behav = join(dir_root, 'behav_data')

In [3]:
def convert_ID(ID):
    ##################   ##################
    #  1  2  3  4  5 #   #        2       #
    #  6  7  8  9 10 #   #        1       #
    # 11 12 13 14 15 # = # -2 -1  0  1  2 #
    # 16 17 18 19 20 #   #       -1       #
    # 21 22 23 24 25 #   #       -2       #
    ##################   ##################
    x = np.kron(np.ones(5),np.arange(-2,3)).astype(int)
    y = np.kron(np.arange(2,-3,-1),np.ones(5)).astype(int)
    pos = np.array((x[ID-1],y[ID-1]))
    return pos

In [4]:
def func_AMregressor(datum):
    sec = 5
    nS = int(datum['nSampleTrial'][0][0]) # 5 s * 60 Hz = 300 samples
    assert sec*60==nS

    ntrial = 12
    nblock = 8
#     tpr = datum['nTrialperRun'][0][0]     # the number of trials per run = 1 trial+ 12 trial/block * 8 blocks
    tpr = 97
    assert 1+ntrial*nblock==tpr

    onsettime = datum['LearnTrialStartTime'][0]
    idx_editpoint = [i+1 for i,t in enumerate(onsettime[:-2]) if (onsettime[i]>onsettime[i+1])]
    assert (np.diff(idx_editpoint)==tpr).all() # Are you sure the fact that 145 trials per run?

#     nrun = int(datum['nRun'][0][0])       # the total number of runs = 6 runs
    nrun = 3
    ## target ID
    tmp = datum['targetID'][0]
    targetID = tmp[tmp!=0][:tpr*nrun]    # targetID.shape = 291 trials = 97 trial/run * 3 runs

    ## onset times
    tmp = np.zeros((nrun, tpr), dtype=float)
    for run in range(nrun):
        idx = idx_editpoint[run]
        tmp[run,:] = onsettime[idx:idx+tpr]*0.001
    onsettime=tmp

    ## counting how many times did they hit the target
    hit_or_not = np.zeros((tpr*nrun, nS), dtype=bool) # hit_or_not.shape = (# of trials/run, # if frames/trial)
    for t, ID in enumerate(targetID):
        pos = datum['boxSize']*convert_ID(ID) # r_target = [x_target, y_target]
        ## allXY.shape = (2, 60 Hz * 4 s/trial * 145 trials/run * 6 runs = 208800 frames)
        xy = datum['allXY'][:,nS*t:nS*(t+1)] # r_cursor = [x_cursor, y_cursor]
        ## err.shape = (2, nS)
        err = xy - np.ones((2,nS))*pos.T # dr = r_cursor - r_target
        ## is the cursor in the target box?
        hit_or_not[t,:] = (abs(err[0,:]) <= datum['boxSize']*0.5) & (abs(err[1,:]) <= datum['boxSize']*0.5)

    cnt_hit = hit_or_not.reshape(nrun, tpr, sec, 60).sum(axis=(2,3))

    return onsettime, cnt_hit

In [52]:
group = scipy.io.loadmat(join(dir_behav, 'mrew_errR_30_run1to3_20190626.mat'))

In [69]:
list_fname = sorted(glob(join(dir_behav, 'GA??-fmri.mat')))
for fname in list_fname:
    subj = fname.split('/')[-1].split('-')[0]
    
    datum = scipy.io.loadmat(fname)

    nrun = 3
    sec = 5

    onsettime, cnt_hit = func_AMregressor(datum)
    reward = cnt_hit/(sec*60)
    
    print(subj)
    a = group['%s_mrew_all'%subj][0][:8*3]
    b = np.mean(reward[:,1:].reshape(3,8,12), axis=-1).reshape(-1)
    print(np.sum(a-b))

GA01
-1.1102230246251565e-16
GA02
-1.942890293094024e-16
GA05
-3.2612801348363973e-16
GA07
1.6653345369377348e-16
GA08
3.608224830031759e-16
GA11
-2.42861286636753e-16
GA12
-5.551115123125783e-17
GA13
2.7755575615628914e-17
GA14
-1.3877787807814457e-17
GA15
-3.0531133177191805e-16
GA18
-3.885780586188048e-16
GA19
5.273559366969494e-16
GA20
2.7755575615628914e-17
GA21
-1.0061396160665481e-16
GA23
4.440892098500626e-16
GA26
6.938893903907228e-17
GA27
1.1102230246251565e-16
GA28
1.5265566588595902e-16
GA29
-2.7755575615628914e-17
GA30
-4.996003610813204e-16
GA31
-2.7755575615628914e-16
GA32
0.0
GA33
-4.163336342344337e-17
GA34
3.885780586188048e-16
GA35
1.1102230246251565e-16
GA36
2.7755575615628914e-17
GA37
6.938893903907228e-18
GA38
-1.6653345369377348e-16
GA42
-2.220446049250313e-16
GA44
-1.3877787807814457e-16


In [70]:
list_fname = sorted(glob(join(dir_behav, 'GA??-refmri.mat')))
for fname in list_fname:
    subj = fname.split('/')[-1].split('-')[0]
    
    datum = scipy.io.loadmat(fname)

    nrun = 3
    sec = 5

    onsettime, cnt_hit = func_AMregressor(datum)
    reward = cnt_hit/(sec*60)
    
    print(subj)
    a = group['%s_mrew_all'%subj][0][-8*3:]
    b = np.mean(reward[:,1:].reshape(3,8,12), axis=-1).reshape(-1)
    print(np.sum(a-b))

GA01
-6.661338147750939e-16
GA02
-5.551115123125783e-16
GA05
-3.3306690738754696e-16
GA07
-2.220446049250313e-16
GA08
0.0
GA11
-2.220446049250313e-16
GA12
4.996003610813204e-16
GA13
-5.551115123125783e-16
GA14
-5.551115123125783e-17
GA15
-1.1102230246251565e-16
GA18
8.881784197001252e-16
GA19
-2.220446049250313e-16
GA20
3.3306690738754696e-16
GA21
4.440892098500626e-16
GA23
2.220446049250313e-16
GA26
0.0
GA27
-2.220446049250313e-16
GA28
5.551115123125783e-16
GA29
0.0
GA30
-2.7755575615628914e-16
GA31
2.220446049250313e-16
GA32
-1.1102230246251565e-16
GA33
-1.1102230246251565e-16
GA34
1.1102230246251565e-16
GA35
-4.996003610813204e-16
GA36
4.440892098500626e-16
GA37
5.551115123125783e-16
GA38
-1.1102230246251565e-16
GA42
-6.661338147750939e-16
GA44
-7.771561172376096e-16
